In [1]:
import pandas    as pd
import seaborn   as sea 
import matplotlib.pyplot as plt
from pyspark.sql       import SparkSession, conf
from pyspark.sql       import functions as f
from pyspark.sql.types import *



In [2]:
spark = SparkSession.builder\
    .appName('ProjetoSprint01')\
    .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/01 17:21:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/01 17:21:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<SparkContext master=local[*] appName=ProjetoSprint01>

### Carregando dados usando pyspark
> Defini o tipo do Schema(com o `StructType`) para alguns Conj. dados pois estavam tipando errado o schema, por exemplo, **`geolocation_zip_code_prefix`** que seus atributos são do tipo string contedo só numero de caracteres, mas estavam sendo colocados como do tipo `integer`.

In [3]:
# olist_customers_dataset

customers     = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','False')\
        .load('dataset/olist_customers_dataset.csv')


geolocation   = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','False')\
        .load('dataset/olist_geolocation_dataset.csv')

# olist_order_payments_dataset
payments      = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/olist_order_payments_dataset.csv')

# olist_orders_dataset
orders        = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/olist_orders_dataset.csv')

# olist_order_items_dataset
order_items   = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/olist_order_items_dataset.csv')

# olist_order_reviews_dataset
order_reviews = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/olist_order_reviews_dataset.csv')

# olist_products_dataset
order_product = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/olist_products_dataset.csv')

# olist_sellers_dataset
sellers      = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','False')\
        .load('dataset/olist_sellers_dataset.csv')

# product_category_dataset
product_cat  = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/product_category_name_translation.csv')

Criando uma visão temporaria , pois usarei o sql do spark para fazer as questões e melhor de entender oq estou fazendo

In [4]:
customers.createOrReplaceTempView('customers')
geolocation.createOrReplaceTempView('geolocation')
orders.createOrReplaceTempView('orders')
order_items.createOrReplaceTempView('order_items')
order_product.createOrReplaceTempView('order_product')
order_reviews.createOrReplaceTempView('order_reviews')
payments.createOrReplaceTempView('payments')
sellers.createOrReplaceTempView('sellers')
product_cat.createOrReplaceTempView('product_cat')

## Analise

Verificação de dados Nulos, fazerei uma função que recebe como parametros string(`nome do conj. dados`) e data(`conj. de dados em si`) ele irá me printar quantidade de valores nulos por coluna 

In [5]:
def VerificaValoresNulos(string, data):
    selecao = data.select(
        [ f.count( f.when(f.isnull(col), col)).alias(col)  for col in data.columns ]
    ).toPandas().T
    selecao.columns = ['Quantidade de valores Nulos']
    print(f'Para o conjuto de dados {string.title()} , com {data.count()} linhas')
    display(selecao)

# Order

In [2]:
VerificaValoresNulos('orders', orders)

NameError: name 'VerificaValoresNulos' is not defined

In [ ]:
orders.toPandas().isna().nunique()

order_id                         1
customer_id                      1
order_status                     1
order_purchase_timestamp         1
order_approved_at                2
order_delivered_carrier_date     2
order_delivered_customer_date    2
order_estimated_delivery_date    1
dtype: int64

In [ ]:
groupOrderStatus = orders\
    .groupBy(f.col('order_status').alias('Status'))\
        .count()\
    .orderBy(f.desc('count'))\
    .withColumn(
        '%', f.round( (( f.col('count')/orders.count() )*100) ,4)
        )

groupOrderStatus.show()

+-----------+-----+-------+
|     Status|count|      %|
+-----------+-----+-------+
|  delivered|96478|97.0203|
|    shipped| 1107| 1.1132|
|   canceled|  625| 0.6285|
|unavailable|  609| 0.6124|
|   invoiced|  314| 0.3158|
| processing|  301| 0.3027|
|    created|    5|  0.005|
|   approved|    2|  0.002|
+-----------+-----+-------+



In [ ]:
orders.count()

99441